In [1]:
from keras.models import *
from keras.callbacks import *
from keras import backend as K
from data import *
import cv2
import argparse
import h5py
import matplotlib.pyplot as plt
from time import sleep
import numpy
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
from termcolor import colored
numpy.set_printoptions(threshold=numpy.nan)
from modelNIHDenseNet import *
from pprint import pprint as pp
from keras.preprocessing.image import ImageDataGenerator
#from sklearn.utils import class_weight as cw
import tensorflow as tf
import random
import time
import json
import pickle

#FLAG_DATASET_PATH = 'data'
FLAG_DATASET_PATH = '/mnt/DataStorage/HumanProtein/'
os.environ["CUDA_VISIBLE_DEVICES"]="0"
# set tensorflow don't use total GPU memory
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

CAM_LAYER_NAME= {'vgg16':'block5_conv3', 'resnet50':'activation_49', 'nasnet_mobile':'normal_concat_12', 'densenet':'normal_concat_12'}
FLAG_MODEL_CHOSEN = 'densenet'

FLAG_TRAIN_BATCH_SIZE = 9
FLAG_TRAIN_TOTAL_EPOCH = 200
FLAG_CLASS_WEIGHT_SMOOTHING = 0.4
FLAS_HP_IMAGE_SIZE = 512

FLAG_DATA_MODE_INDEX = 0
#FLAG_NORMAL_MODE = 0, FLAG_BINARY_MODE = 1, FLAG_ONLY_DISEASE_MODE = 2
#current, only support normal mode
FLAG_DATA_MODE = ['FLAG_NORMAL_MODE', 'FLAG_BINARY_MODE']

LAST_MODEL_NAME = 'weights.epoch.sub.49-0.66.h5'
LAST_MODEL_PATH = os.path.join('model', FLAG_MODEL_CHOSEN, LAST_MODEL_NAME)

FLAG_DATASET_IMAGE = os.path.join(FLAG_DATASET_PATH, 'images')
Data_MODE = 'FULL'

FLAG_TEST_DATASET_IMAGE = os.path.join(FLAG_DATASET_PATH, 'test')

sample_csv_path = os.path.join(FLAG_DATASET_PATH, 'sample_submission.csv')
submission_csv_path = os.path.join(FLAG_DATASET_PATH, 'submission.csv')

TAGS = 28

# set tensorflow don't use total GPU memory
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

float_formatter = lambda x: "%.5f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

K.set_learning_phase(0)
model = get_model(TAGS, FLAS_HP_IMAGE_SIZE)
model.load_weights(LAST_MODEL_PATH)


Using TensorFlow backend.


new layer name =  image_input  layer.trainable =  False
new layer name =  zero_padding2d_1  layer.trainable =  True
new layer name =  conv1/conv  layer.trainable =  True
new layer name =  conv1/bn  layer.trainable =  True
new layer name =  conv1/relu  layer.trainable =  True
new layer name =  zero_padding2d_2  layer.trainable =  True
new layer name =  pool1  layer.trainable =  True
new layer name =  conv2_block1_0_bn  layer.trainable =  True
new layer name =  conv2_block1_0_relu  layer.trainable =  True
new layer name =  conv2_block1_1_conv  layer.trainable =  True
new layer name =  conv2_block1_1_bn  layer.trainable =  True
new layer name =  conv2_block1_1_relu  layer.trainable =  True
new layer name =  conv2_block1_2_conv  layer.trainable =  True
new layer name =  conv2_block1_concat  layer.trainable =  True
new layer name =  conv2_block2_0_bn  layer.trainable =  True
new layer name =  conv2_block2_0_relu  layer.trainable =  True
new layer name =  conv2_block2_1_conv  layer.trainable

new layer name =  conv5_block1_1_bn  layer.trainable =  True
new layer name =  conv5_block1_1_relu  layer.trainable =  True
new layer name =  conv5_block1_2_conv  layer.trainable =  True
new layer name =  conv5_block1_concat  layer.trainable =  True
new layer name =  conv5_block2_0_bn  layer.trainable =  True
new layer name =  conv5_block2_0_relu  layer.trainable =  True
new layer name =  conv5_block2_1_conv  layer.trainable =  True
new layer name =  conv5_block2_1_bn  layer.trainable =  True
new layer name =  conv5_block2_1_relu  layer.trainable =  True
new layer name =  conv5_block2_2_conv  layer.trainable =  True
new layer name =  conv5_block2_concat  layer.trainable =  True
new layer name =  conv5_block3_0_bn  layer.trainable =  True
new layer name =  conv5_block3_0_relu  layer.trainable =  True
new layer name =  conv5_block3_1_conv  layer.trainable =  True
new layer name =  conv5_block3_1_bn  layer.trainable =  True
new layer name =  conv5_block3_1_relu  layer.trainable =  True
ne

In [2]:
data_name = 'data_' + Data_MODE + '_0_' + FLAG_DATA_MODE[FLAG_DATA_MODE_INDEX] + '.h5'
model_h5_path = os.path.join(FLAG_DATASET_PATH, data_name)
data_model = tables.open_file(model_h5_path, mode='a')
print('data size = ', len(data_model.root.data))

train_size = int(len(data_model.root.tag)*0.8)

validation_x = data_model.root.data[train_size:]
validation_y = data_model.root.tag[train_size:]

tf_session = K.get_session()
validation_y_pred = model.predict(
    validation_x, 
    batch_size=4, 
    verbose=1,
    )
#y = y.astype(np.float32)

dice_score = f1(validation_y, validation_y_pred)
print('f1 = ', dice_score.eval(session=tf_session))

data size =  31072


ValueError: Error when checking input: expected image_input to have shape (512, 512, 4) but got array with shape (224, 224, 4)

In [ ]:
tf_session = K.get_session()
df_train = pd.read_csv(sample_csv_path)
for row in tqdm(df_train.values[:]):
    f = row[0]
    tag_files = [FLAG_TEST_DATASET_IMAGE+'/{}'.format(f)+'_blue.png', FLAG_TEST_DATASET_IMAGE+'/{}'.format(f)+'_green.png', FLAG_TEST_DATASET_IMAGE+'/{}'.format(f)+'_blue.png', FLAG_TEST_DATASET_IMAGE+'/{}'.format(f)+'_yellow.png']
    new_image = np.zeros((0, FLAS_HP_IMAGE_SIZE, FLAS_HP_IMAGE_SIZE))
    for file in tag_files:
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        img = img / 255.
        img = np.expand_dims(img, axis=0)
        new_image = np.vstack((new_image, img))        
    
    new_image = np.transpose(new_image, (1,2,0))
    new_image = np.expand_dims(new_image, axis=0)
        
    validation_y_pred = model.predict(
        new_image, 
        batch_size=1, 
        verbose=1,
    )
    
    pred_index = []
    for i, v in enumerate(validation_y_pred[0]):
        if v>0.5:
            pred_index.append(str(i)+' ')
    
    row[1] = pred_index
    print(row[1])

df.to_csv(submission_csv_path, encoding='utf-8', index=False)

  0%|          | 0/11702 [00:00<?, ?it/s]

1/1 [==============================] - 0s 48ms/step
['2 ', '3 ', '7 ', '8 ', '9 ', '10 ', '15 ', '16 ', '17 ', '20 ', '23 ', '27 ']
1/1 [==============================] - 0s 48ms/step


  0%|          | 2/11702 [00:00<16:33, 11.77it/s]

['1 ', '3 ', '4 ', '5 ', '6 ', '11 ', '14 ', '17 ', '18 ', '25 ']
1/1 [==============================] - 0s 48ms/step
['0 ', '1 ', '5 ', '14 ', '17 ', '18 ', '19 ', '21 ', '25 ']
1/1 [==============================] - 0s 47ms/step


  0%|          | 4/11702 [00:00<16:28, 11.83it/s]

...

['1 ', '6 ', '8 ', '11 ', '14 ', '17 ', '20 ', '23 ']
1/1 [==============================] - 0s 44ms/step
['7 ', '15 ', '17 ', '21 ', '23 ']
1/1 [==============================] - 0s 44ms/step


  2%|▏         | 258/11702 [00:20<15:06, 12.63it/s]

['1 ', '2 ', '3 ', '5 ', '8 ', '15 ', '18 ', '19 ', '24 ', '26 ', '27 ']
1/1 [==============================] - 0s 45ms/step
['7 ', '23 ']
1/1 [==============================] - 0s 43ms/step


  2%|▏         | 260/11702 [00:20<15:06, 12.62it/s]

['4 ', '12 ', '13 ', '14 ', '21 ', '22 ', '25 ', '26 ']
